In [ ]:
# --- Environment & installs ---
!pip install datasets==2.16.0
!pip install huggingface-hub==0.20.0
!apt-get install -y libsox-dev
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install causal-conv1d==1.4.0 mamba-ssm==2.2.2 scikit-learn tensorboard

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Fou

In [ ]:
# =========================================================
# Keyword-Spotting (Google Speech Commands v0.02) with Mamba
# Front-end: MFCC (F=40) -> Linear(40 -> d_model) -> Mamba * L -> Classifier
# =========================================================

from __future__ import annotations
import json, os, random, math, time
from pathlib import Path
from typing import Tuple, Dict, List

import torch, torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from datasets import load_dataset
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

from mamba_ssm import Mamba

# ---------------------------------------------------------
# 0) Repro & device
# ---------------------------------------------------------
def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_amp = (device.type == "cuda")
set_seed(42)

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/dis

Mounted at /content/drive


In [ ]:
# ---------------------------------------------------------
# 1) Waveform-level augmentation (shift + noise)
# ---------------------------------------------------------
class Augment:
    def __init__(self,
                 stretch: Tuple[float,float]=(1.0,1.0),
                 shift_ms: int = 100,
                 noise: Tuple[float,float]=(0.,0.05),
                 sr: int = 16_000):
        self.stretch = stretch
        self.shift   = int(shift_ms * sr / 1000)
        self.noise   = noise
        self.sr      = sr

    def _shift(self, x: torch.Tensor):
        if self.shift == 0:
            return x
        s = int(torch.randint(-self.shift, self.shift + 1, ()).item())
        if s == 0:
            return x
        return (F.pad(x, (s, 0))[:, :-s] if s > 0 else F.pad(x, (0, -s))[:, -s:])

    def __call__(self, wav: torch.Tensor):
        squeezed = False
        if wav.dim() == 1:
            wav = wav.unsqueeze(0)
            squeezed = True
        if self.stretch != (1.0, 1.0):
            factor = float(torch.empty(()).uniform_(*self.stretch))
            if abs(factor - 1.0) > 1e-3:
                wav, _ = torchaudio.sox_effects.apply_effects_tensor(
                    wav, self.sr, [["tempo", f"{factor}"]]
                )
        wav = self._shift(wav)
        if self.noise[1] > 0:
            sigma = float(torch.empty(()).uniform_(*self.noise))
            if sigma > 0:
                wav = wav + sigma * torch.randn_like(wav)
        return wav.squeeze(0) if squeezed else wav

# ---------------------------------------------------------
# 2) Front-end: MFCC or Mel with SpecAug AFTER log (if mel)
#    For MFCC we mask on the MFCCs (computed from log-mel internally).
# ---------------------------------------------------------
class WaveToSpec:
    def __init__(self,
                 feature_type: str = "mfcc",     # "mfcc" or "mel"
                 sample_rate: int = 16_000,
                 n_fft: int = 2048,
                 hop_length: int = 256,
                 n_mels: int = 128,
                 n_mfcc: int = 40,
                 top_db: int | None = 80,
                 apply_mask: bool = True,
                 freq_mask_param: int = 3,
                 time_mask_param: int = 12):
        self.feature_type = feature_type.lower(); assert self.feature_type in {"mel","mfcc"}
        self.apply_mask = apply_mask

        if self.feature_type == "mel":
            self.spec = T.MelSpectrogram(sample_rate, n_fft=n_fft, hop_length=hop_length,
                                         n_mels=n_mels, power=2.0)
            self.to_db = T.AmplitudeToDB(stype="power", top_db=top_db)
            self.freq_mask = T.FrequencyMasking(freq_mask_param) if apply_mask else None
            self.time_mask = T.TimeMasking(time_mask_param) if apply_mask else None
            self.n_out = n_mels
        else:
            self.spec = T.MFCC(sample_rate, n_mfcc=n_mfcc,
                                melkwargs=dict(n_fft=n_fft, hop_length=hop_length, n_mels=n_mels))
            # MFCC branch does not use AmplitudeToDB directly; MFCC uses log-mel inside.
            self.to_db = None
            self.freq_mask = T.FrequencyMasking(freq_mask_param) if apply_mask else None
            self.time_mask = T.TimeMasking(time_mask_param) if apply_mask else None
            self.n_out = n_mfcc

    def __call__(self, wav: torch.Tensor) -> torch.Tensor:
        if wav.dim() == 1:
            wav = wav.unsqueeze(0)            # [1, T]
        feats = self.spec(wav)                # mel: [1, M, T], mfcc: [1, C, T]

        if self.feature_type == "mel":
            feats = self.to_db(feats.clamp(min=1e-10))  # --- LOG ---
            if self.apply_mask:
                feats = self.freq_mask(feats)
                feats = self.time_mask(feats)
        else:
            # MFCC path: apply masks on MFCCs (post log-mel inside MFCC)
            if self.apply_mask:
                feats = self.freq_mask(feats)
                feats = self.freq_mask(feats)
                feats = self.time_mask(feats)
                feats = self.time_mask(feats)

        return feats                           # [1, F, T]

In [ ]:
# ---------------------------------------------------------
# 3) Dataset wrapper with dataset-level normalization
# ---------------------------------------------------------
class SpeechCommands(Dataset):
    def __init__(self, hf_split, aug: Augment | None, frontend: WaveToSpec,
                 wav_len: int = 16_000, mean: float = 0.0, std: float = 1.0):
        self.ds, self.aug, self.front = hf_split, aug, frontend
        self.wav_len = wav_len
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        wav = torch.from_numpy(sample["audio"]["array"]).float()

        if wav.numel() < self.wav_len:
            wav = F.pad(wav, (0, self.wav_len - wav.numel()))
        else:
            wav = wav[: self.wav_len]

        if self.aug:
            wav = self.aug(wav)

        feats = self.front(wav)                              # [1, F, T]
        feats = (feats - self.mean) / (self.std + 1e-6)      # normalize
        feats = feats.squeeze(0).transpose(0, 1)             # [T, F]
        return feats, sample["label"]

In [ ]:
# ---------------------------------------------------------
# 4) Helper functions + stats
# ---------------------------------------------------------
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    feats, lbls = zip(*batch)
    lens = torch.tensor([f.size(0) for f in feats], dtype=torch.long)
    feats_padded = pad_sequence(feats, batch_first=True, padding_value=0.0)  # [B, T_max, F]
    return feats_padded, torch.tensor(lbls), lens

@torch.no_grad()
def compute_dataset_stats(ds, frontend, wav_len=16_000):
    feats_all = []
    for sample in ds:
        wav = torch.from_numpy(sample["audio"]["array"]).float()
        if wav.numel() < wav_len:
            wav = F.pad(wav, (0, wav_len - wav.numel()))
        else:
            wav = wav[: wav_len]
        feats = frontend(wav).squeeze(0).transpose(0, 1)  # [T, F]
        feats_all.append(feats)
    feats_all = torch.cat(feats_all, dim=0)
    return feats_all.mean().item(), feats_all.std().item()

In [ ]:
# ---------------------------------------------------------
# 5) Model: Linear(40→d_model) + Mamba×L + classifier
# ---------------------------------------------------------
class MambaKWS(nn.Module):
    def __init__(self, num_classes: int,
                 d_model=256, d_state=16, expand=2, n_layers=8, feature_dim=40, p_drop=0.1):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Linear(feature_dim, d_model),
            nn.LayerNorm(d_model),
            nn.SiLU(),
            nn.Dropout(p_drop)
        )
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                "norm": nn.LayerNorm(d_model),
                "mamba": Mamba(d_model=d_model, d_state=d_state, expand=expand),
                "dropout": nn.Dropout(max(0.02, 0.05 - (i * 0.005)))
            }) for i in range(n_layers)
        ])
        self.pre_classifier_norm = nn.LayerNorm(d_model)
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(d_model, d_model // 2),
            nn.SiLU(),
            nn.Dropout(0.05),
            nn.Linear(d_model // 2, num_classes)
        )

    def forward(self, x, lengths: torch.Tensor | None = None):  # x: [B, T, F]
        x = self.proj(x)                                        # [B, T, d_model]
        for blk in self.blocks:
            residual = x
            x = blk["norm"](x)
            x = blk["mamba"](x)
            x = blk["dropout"](x)
            x = residual + x

        x = self.pre_classifier_norm(x)

        # Mask-aware mean pooling over time (no time downsampling here)
        if lengths is not None:
            Tprime = x.size(1)
            mask = (torch.arange(Tprime, device=x.device)[None, :] < lengths[:, None]).float()  # [B, T]
            x_sum = (x * mask.unsqueeze(-1)).sum(dim=1)                                          # [B, d_model]
            denom = mask.sum(dim=1).clamp(min=1.0).unsqueeze(-1)                                 # [B, 1]
            pooled = x_sum / denom
        else:
            pooled = x.mean(dim=1)

        return self.classifier(pooled)                        # [B, C]

# ---------------------------------------------------------
# 6) Evaluation helpers (loss, acc, macro-F1)
# ---------------------------------------------------------
@torch.no_grad()
def evaluate(model, loader, device, criterion):
    model.eval()
    tot = correct = loss_sum = 0
    all_pred, all_true = [], []
    for xb, yb, lb in loader:
        xb, yb, lb = xb.to(device), yb.to(device), lb.to(device)
        logits = model(xb, lengths=lb)
        loss = criterion(logits, yb)
        loss_sum += loss.item() * xb.size(0)
        preds = logits.argmax(1)
        correct += (preds == yb).sum().item()
        tot += xb.size(0)
        all_pred.append(preds.detach().cpu())
        all_true.append(yb.detach().cpu())
    all_pred = torch.cat(all_pred).numpy()
    all_true = torch.cat(all_true).numpy()
    macro_f1 = f1_score(all_true, all_pred, average='macro')
    return loss_sum / tot, 100 * correct / tot, macro_f1

In [ ]:
# ---------------------------------------------------------
# 7) Warmup + Cosine scheduler factory (per-batch)
# ---------------------------------------------------------
def make_warmup_cosine(total_steps: int, warmup_steps: int):
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return max(0.003, 0.5 * (1.0 + math.cos(math.pi * progress)))
    return lr_lambda

In [ ]:
# ---------------------------------------------------------
# 8) Main training loop over the config grid + TensorBoard
# ---------------------------------------------------------
def run_experiment(cfg: Dict, ds_splits, global_outdir="checkpoints"):
    d_model   = cfg["d_model"]
    n_layers  = cfg["n_layers"]
    epochs    = cfg["epochs"]
    base_lr   = cfg.get("base_lr", 0.001)
    weight_decay = cfg.get("weight_decay", 0.1)
    label_smoothing = cfg.get("label_smoothing", 0.1)
    d_state   = cfg.get("d_state", 16)
    expand    = cfg.get("expand", 2)

    # Unpack datasets/loaders prepared outside
    train_dl, train_eval_dl, val_dl, n_classes = ds_splits

    # Model / optim / sched
    model = MambaKWS(n_classes, d_model=d_model, d_state=d_state, expand=expand, n_layers=n_layers).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    opt = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay, betas=(0.9, 0.999))

    steps_per_epoch = len(train_dl)
    total_steps   = steps_per_epoch * epochs
    warmup_steps  = int(0.12 * total_steps)  # 12% warmup
    sched = torch.optim.lr_scheduler.LambdaLR(opt, make_warmup_cosine(total_steps, warmup_steps))

    # TensorBoard writer
    run_name = f"d{d_model}_L{n_layers}"
    writer = SummaryWriter(log_dir=f"runs/mamba_kws/{run_name}")
    writer.add_text("hparams", json.dumps(cfg, indent=2))

    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    best_val_acc, best_path = 0.0, Path(f"{global_outdir}/{run_name}_best.pt")
    best_path.parent.mkdir(parents=True, exist_ok=True)

    global_step = 0
    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = correct = total = 0.0
        pbar = tqdm(train_dl, desc=f"[{run_name}] Epoch {epoch:02d}")

        for xb, yb, lb in pbar:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            lb = lb.to(device, non_blocking=True)

            with torch.amp.autocast(device_type=device.type,
                                    dtype=torch.bfloat16 if (use_amp and torch.cuda.is_bf16_supported()) else torch.float16,
                                    enabled=use_amp):
                if torch.isnan(xb).any():
                    xb = torch.nan_to_num(xb, nan=0.0)
                logits = model(xb, lengths=lb)
                loss = criterion(logits, yb)

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            scaler.step(opt)
            scaler.update()
            sched.step()
            global_step += 1

            with torch.no_grad():
                pred = logits.argmax(1)
                correct += (pred == yb).sum().item()
                total += yb.size(0)
                running_loss += loss.item() * yb.size(0)

            pbar.set_postfix(train_loss=f"{running_loss/max(1,total):.3f}",
                             train_acc=f"{100*correct/max(1,total):.1f}%",
                             lr=f"{opt.param_groups[0]['lr']:.2e}")

        # --- Epoch end: log train metrics on NO-AUG train split, and val ---
        tr_loss, tr_acc, tr_f1 = evaluate(model, train_eval_dl, device, criterion)
        val_loss, val_acc, val_f1 = evaluate(model, val_dl, device, criterion)

        writer.add_scalar("train/loss", tr_loss, epoch)
        writer.add_scalar("train/acc",  tr_acc,  epoch)
        writer.add_scalar("train/macro_f1", tr_f1, epoch)
        writer.add_scalar("val/loss",   val_loss, epoch)
        writer.add_scalar("val/acc",    val_acc,  epoch)
        writer.add_scalar("val/macro_f1", val_f1, epoch)
        writer.add_scalar("lr", opt.param_groups[0]['lr'], epoch)

        print(f"[{run_name}] Epoch {epoch:02d} — "
              f"Train {tr_acc:.2f}% (loss {tr_loss:.3f}, F1 {tr_f1:.4f}) | "
              f"Val {val_acc:.2f}% (loss {val_loss:.3f}, F1 {val_f1:.4f})")

        # Save best-by-val-acc
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_path)
            print(f"[{run_name}] ** Saved new BEST ** (val_acc={best_val_acc:.2f}%) → {best_path}")

    # Save last
    last_path = Path(f"{global_outdir}/{run_name}_last.pt")
    torch.save(model.state_dict(), last_path)
    writer.add_text("checkpoints", f"best={str(best_path)} | last={str(last_path)}")
    writer.close()

    return {"run": run_name, "best_val_acc": best_val_acc,
            "best_ckpt": str(best_path), "last_ckpt": str(last_path)}

In [ ]:
# ---------------------------------------------------------
# 9) Data prep (MFCC - Masks only on train)
# ---------------------------------------------------------
if __name__ == "__main__":
    # Load HF dataset
    ds = load_dataset("google/speech_commands", "v0.02")

    n_classes = len(ds["train"].features["label"].names)
    sr = 16_000

    # Front-ends
    frontend_stats = WaveToSpec(feature_type="mfcc", n_mfcc=40, n_mels=128,
                                apply_mask=False)
    frontend_train = WaveToSpec(feature_type="mfcc", n_mfcc=40, n_mels=128,
                                apply_mask=True, freq_mask_param=4, time_mask_param=15)
    frontend_eval  = WaveToSpec(feature_type="mfcc", n_mfcc=40, n_mels=128,
                                apply_mask=False)

    # Waveform augmentations (train only)
    aug = Augment(shift_ms=100, noise=(0., 0.01), stretch=(0.95, 1.05))

    # Dataset-level normalization stats (computed on train, no aug)
    print("Computing dataset stats (MFCC, no masks)...")
    train_mean, train_std = compute_dataset_stats(ds["train"], frontend_stats)
    print(f"Stats — mean={train_mean:.4f}, std={train_std:.4f}")

    # Datasets
    train_ds       = SpeechCommands(ds["train"], aug,  frontend_train, mean=train_mean, std=train_std)
    train_eval_ds  = SpeechCommands(ds["train"], None, frontend_eval,  mean=train_mean, std=train_std)  # no aug
    val_ds         = SpeechCommands(ds["validation"], None, frontend_eval, mean=train_mean, std=train_std)

    # Loaders
    dl_kwargs = dict(batch_size=128, num_workers=2, pin_memory=True,
                     persistent_workers=True, collate_fn=collate_fn)
    train_dl      = DataLoader(train_ds, shuffle=True,  **dl_kwargs)
    train_eval_dl = DataLoader(train_eval_ds, shuffle=False, **dl_kwargs)
    val_dl        = DataLoader(val_ds, shuffle=False, **dl_kwargs)

    ds_splits = (train_dl, train_eval_dl, val_dl, n_classes)

    # -----------------------------------------------------
    # Grid: check configurations
    # -----------------------------------------------------
    outdir = "checkpoints"
    os.makedirs(outdir, exist_ok=True)

    configs = []
    for d_model in [128]:
        for n_layers in [10]:
            configs.append({
                "d_model": d_model,
                "n_layers": n_layers,
                "epochs": 100,
                "base_lr": 0.001,
                "weight_decay": 0.1,
                "label_smoothing": 0.1,
                "d_state": 16,
                "expand": 2
            })

    results = []
    for cfg in configs:
        set_seed(42)  # keep runs comparable
        res = run_experiment(cfg, ds_splits, global_outdir=outdir)
        results.append(res)

    print("\n=== Summary ===")
    for r in results:
        print(r)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1429: FutureWarning: The repository for google/speech_commands contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/speech_commands
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets

Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4890 [00:00<?, ? examples/s]

Computing dataset stats (MFCC, no masks)...
Stats — mean=-5.8153, std=47.6515


...

[d128_L10] Epoch 79:   0%|          | 0/663 [00:00<?, ?it/s]

[d128_L10] Epoch 79 — Train 99.06% (loss 0.699, F1 0.9627) | Val 96.41% (loss 0.780, F1 0.9336)


[d128_L10] Epoch 80:   0%|          | 0/663 [00:00<?, ?it/s]

[d128_L10] Epoch 80 — Train 99.23% (loss 0.693, F1 0.9644) | Val 96.34% (loss 0.778, F1 0.9332)


[d128_L10] Epoch 81:   0%|          | 0/663 [00:00<?, ?it/s]

In [ ]:
from tensorboard import notebook

notebook.start("--logdir runs")

In [ ]:
SAVE_DIR = "/content/drive/MyDrive/kws_exports_medium"
!mkdir -p "$SAVE_DIR"

# Zip folders straight to Drive paths
!zip -qr "$SAVE_DIR/checkpoints_expand.zip" /content/checkpoints
!zip -qr "$SAVE_DIR/runs_expand.zip"        /content/runs

# Verify
!ls -lh "$SAVE_DIR"